<a href="https://colab.research.google.com/github/aemosdaeva/Demo1/blob/main/Copia_de_lectura_de_imagen_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pytesseract
!pip install pdf2image
!pip install opencv-python
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils

import pytesseract  # Asegúrate de que no haya espacios sin ruptura antes de 'import'
from PIL import Image, ImageEnhance, ImageFilter
from pdf2image import convert_from_path
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os

# Configurar Tesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

def procesar_imagen(imagen, calidad="alta"):
    if calidad == "alta":
        imagen = imagen.convert("L")
        imagen = ImageEnhance.Contrast(imagen).enhance(2)
        imagen = ImageEnhance.Sharpness(imagen).enhance(2)
    elif calidad == "media":
        imagen = imagen.convert("L")
        imagen = ImageEnhance.Contrast(imagen).enhance(1.5)
    elif calidad == "baja":
        imagen = imagen.convert("L")
    return imagen

def leer_imagen(archivo_path, calidad="alta"):
    imagen = Image.open(archivo_path)
    imagen = procesar_imagen(imagen, calidad)
    texto = pytesseract.image_to_string(imagen)
    return texto

def leer_pdf(archivo_path, calidad="alta"):
    paginas = convert_from_path(archivo_path)
    texto = ""
    for pagina in paginas:
        pagina_procesada = procesar_imagen(pagina, calidad)
        texto += pytesseract.image_to_string(pagina_procesada)
    return texto

def leer_documento(archivo_path, calidad="alta"):
    if archivo_path.lower().endswith((".jpg", ".jpeg", ".png")):
        return leer_imagen(archivo_path, calidad)
    elif archivo_path.lower().endswith(".pdf"):
        return leer_pdf(archivo_path, calidad)
    else:
        return "Formato de archivo no soportado."


def cargar_datos():
    # Aquí deberías cargar y procesar tus imágenes y etiquetas
    X = []  # Lista de características
    y = []  # Lista de etiquetas (caracteres)

    # Carga la imagen directamente
    imagen = cv2.imread("/content/drive/My Drive/cedula_1144048177.jpg", cv2.IMREAD_GRAYSCALE)

    # ... (resto del procesamiento de la imagen, por ejemplo, redimensionar, aplanar) ...

    X.append(imagen)
    y.append("etiqueta_de_la_imagen")  # Reemplaza con la etiqueta correcta

    return np.array(X), np.array(y)


def mejorar_ocr(imagen):
    # Dividir la imagen en caracteres individuales
    # Supongamos que ya tienes una función para esto
    caracteres = dividir_imagen_en_caracteres(imagen)  # Debes definir esta función

    texto = ""
    for caracter in caracteres:
        caracter = cv2.resize(caracter, (20, 20)).flatten().reshape(1, -1)
        prediccion = modelo.predict(caracter)  # 'modelo' debe estar definido y entrenado
        texto += prediccion[0]

    return texto

# Ruta del archivo en Google Drive
archivo = "/content/drive/My Drive/cedula_1144048177.jpg"

# Leer el documento con diferentes calidades
texto_alta = leer_documento(archivo, calidad="alta")
print("Texto en calidad alta:")
print(texto_alta)

texto_media = leer_documento(archivo, calidad="media")
print("Texto en calidad media:")
print(texto_media)

texto_baja = leer_documento(archivo, calidad="baja")
print("Texto en calidad baja:")
print(texto_baja)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Mounted at /content/drive
Texto en calidad alta:
REPUBLICA DE COLOMBIA
IDENTIFICACION PERSONAL.
CEDULA DE CIUDADANIA
NUMENO 1.144,048.177
MARTINEZ AMAYA

\PELLIDOS

MARIA MONICA

NOMARES

 

Texto en calidad media:
REPUBLICA DE COLOMBIA
IDENTIFICACION PERSONAL. —
CEDULA DE CIUDADANIA
vuweno 1.144,048.177
MARTINEZ AMAYA

PELLIDOS

MARIA MONICA

BM og 5
a .

NOMARES

 

Texto en calidad baja:
REPUBLICA DE COLOMBIA.
IDENTIFICACION PERSONAL.
CEDULA DE CIUDADANIA

wiveno 1.144,048.177
MARTINEZ AMAYA

PELLIDOS

MARIA MONICA

NOMARES

 

